In [0]:
%sql
select * from datamodelling_vasu.silver.silver_table

In [0]:
%sql
SELECT DISTINCT(customer_id),
       CustomerName,
       customer_email,
       Country
from datamodelling_vasu.silver.silver_table
  order by customer_id;

In [0]:
%sql
SELECT *,  row_number() over(order by customer_id) as DimCustomerKey
from (
  SELECT DISTINCT(customer_id),
       CustomerName,
       customer_email,
       Country
  from datamodelling_vasu.silver.silver_table
  order by customer_id);

In [0]:
%sql
CREATE OR REPLACE TABLE datamodelling_vasu.gold.DimCustomers1
AS
SELECT *,  row_number() over(order by customer_id) as DimCustomerKey
from
    (SELECT DISTINCT(customer_id),
       CustomerName,
       customer_email
    from 
       datamodelling_vasu.silver.silver_table
    order by customer_id)

In [0]:
%sql
select * from datamodelling_vasu.gold.DimCustomers1

In [0]:
%sql
CREATE OR REPLACE TABLE datamodelling_vasu.gold.DimCustomers
AS
WITH REMOVE_DUPLICATES
  (SELECT DISTINCT(customer_id),
       CustomerName,
       customer_email
  from datamodelling_vasu.silver.silver_table
    order by customer_id)
SELECT *,  row_number() over(order by customer_id) as DimCustomerKey
from REMOVE_DUPLICATES;

In [0]:
%sql
select * from datamodelling_vasu.gold.dimcustomers;

In [0]:
%sql
SELECT DISTINCT(product_id),
       product_name,
       product_category
  from datamodelling_vasu.silver.silver_table
  order by product_id

In [0]:
%sql
SELECT *,  row_number() over(order by product_id) as DimProductKey
from (
  SELECT DISTINCT(product_id),
       product_name,
       product_category
  from datamodelling_vasu.silver.silver_table
  order by product_id);

In [0]:
%sql
CREATE OR REPLACE TABLE datamodelling_vasu.gold.DimProducts
AS
WITH REMOVE_DUPLICATE_PRODUCTS
  (
    SELECT DISTINCT(product_id),
       product_name,
       product_category
    from datamodelling_vasu.silver.silver_table
  )
SELECT *,
       row_number() over(order by product_id) as DimProductKey
from REMOVE_DUPLICATE_PRODUCTS

In [0]:
%sql
SELECT * FROM datamodelling_vasu.gold.DimProducts;

In [0]:
%sql
CREATE OR REPLACE TABLE datamodelling_vasu.gold.DimRegions
AS
WITH REMOVE_DUPLICATE_Countries
  (
    SELECT DISTINCT(country)
    from datamodelling_vasu.silver.silver_table
  )
SELECT *,
       row_number() over(order by country) as DimRegionKey
from remove_duplicate_countries

In [0]:
%sql
select * from datamodelling_vasu.gold.DimRegions

In [0]:
%sql
CREATE OR REPLACE TABLE datamodelling_vasu.gold.DimPayments
AS
WITH REMOVE_DUPLICATE_PAYMENTS
  (
    SELECT DISTINCT(payment_type)
    from datamodelling_vasu.silver.silver_table
  )
SELECT *,
       row_number() over(order by payment_type) as DimPAYMENTKey
from REMOVE_DUPLICATE_PAYMENTS

In [0]:
%sql
select * from datamodelling_vasu.gold.DimPayments

### Dim Sales

In [0]:
spark.sql("select * from datamodelling_vasu.silver.silver_table").columns

In [0]:
%sql
CREATE OR REPLACE TABLE datamodelling_vasu.gold.DimSalesKey
AS
WITH REMOVE_DUPLICATE_PAYMENTS
  (
    SELECT DISTINCT(order_id),
       order_date,
       customer_id,
       CustomerName,
       customer_email,
       product_id,
       product_name,
       product_category,
       payment_type,
       country,
       last_updated,
       Processed_Date
    from datamodelling_vasu.silver.silver_table
  )
SELECT *,
       row_number() over(order by order_id) as DimSalesKey 
       from REMOVE_DUPLICATE_PAYMENTS

In [0]:
%sql
SELECT *
FROM
    datamodelling_vasu.gold.DimSalesKey

#### Fact Table

In [0]:
%sql
CREATE OR REPLACE TABLE datamodelling_vasu.gold.FactSales
AS
select 
  S.DimSalesKey,
  C.DimCustomerkey,
  P.DimProductKey,
  R.DimRegionKey,
  PY.DimPaymentKey,
  F.quantity,
  F.unit_price
from
  datamodelling_vasu.silver.silver_table F
  left join datamodelling_vasu.gold.DimCustomers C on F.customer_id = C.customer_id
  Left join datamodelling_vasu.gold.DimProducts P on F.product_id = P.product_id
  Left join datamodelling_vasu.gold.DimRegions R on F.country = R.country
  Left join datamodelling_vasu.gold.DimPayments PY on F.payment_type = PY.payment_type
  left join datamodelling_vasu.gold.DimSalesKey S on F.order_id = S.order_id